In [1]:
import pandas as pd
import altair as alt
import numpy as np
from sklearn import set_config
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Simplify working with large datasets in Altair
alt.data_transformers.enable('vegafusion')

# Output dataframes instead of arrays
set_config(transform_output="pandas")

In [2]:
players = pd.read_csv("players.csv")
players

,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,NaN,NaN
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,NaN,NaN
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,NaN,NaN
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,NaN,NaN
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,NaN,NaN
...,...,...,...,...,...,...,...,...,...
191,Amateur,True,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17,NaN,NaN
192,Veteran,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22,NaN,NaN
193,Amateur,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17,NaN,NaN
194,Amateur,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17,NaN,NaN


In [3]:
players_filtered = players.loc[players["gender"]!="Prefer not to say", ["experience", "gender", "age", "played_hours"]]
players_filtered = players_filtered[players_filtered["played_hours"] != 0.0]
players_filtered

,experience,gender,age,played_hours
0,Pro,Male,9,30.3
1,Veteran,Male,17,3.8
3,Amateur,Female,21,0.7
4,Regular,Male,21,0.1
8,Amateur,Male,17,0.1
...,...,...,...,...
185,Regular,Male,18,0.1
186,Veteran,Female,44,0.1
192,Veteran,Male,22,0.3
194,Amateur,Male,17,2.3


In [4]:
players_filtered["gender"] = players_filtered["gender"].replace({
    "Male" : 0,
    "Female" : 1, 
    "Agender" : 2, 
    "Other": 3, 
    "Non-binary": 4,
    "Two-Spirited": 5
})
players_filtered

/tmp/ipykernel_115/3290860224.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  players_filtered["gender"] = players_filtered["gender"].replace({


,experience,gender,age,played_hours
0,Pro,0,9,30.3
1,Veteran,0,17,3.8
3,Amateur,1,21,0.7
4,Regular,0,21,0.1
8,Amateur,0,17,0.1
...,...,...,...,...
185,Regular,0,18,0.1
186,Veteran,1,44,0.1
192,Veteran,0,22,0.3
194,Amateur,0,17,2.3


In [5]:
players_filtered["experience"] = players_filtered["experience"].replace({
    "Pro" : 0,
    "Amateur" : 1, 
    "Regular" : 2, 
    "Beginner": 3, 
    "Veteran": 4,
})
players_filtered

/tmp/ipykernel_115/3026204355.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  players_filtered["experience"] = players_filtered["experience"].replace({


,experience,gender,age,played_hours
0,0,0,9,30.3
1,4,0,17,3.8
3,1,1,21,0.7
4,2,0,21,0.1
8,1,0,17,0.1
...,...,...,...,...
185,2,0,18,0.1
186,4,1,44,0.1
192,4,0,22,0.3
194,1,0,17,2.3


In [27]:
play_preprocessor = make_column_transformer(
    (StandardScaler(), ["experience", "gender", "age"]),
    verbose_feature_names_out=False,
)
play_preprocessor

players_scaled = play_preprocessor.fit_transform(players_filtered)
players_scaled

,experience,gender,age
0,-1.540191,-0.482243,-1.076360
1,1.414746,-0.482243,-0.410117
3,-0.801457,0.369719,-0.076995
4,-0.062723,-0.482243,-0.076995
8,-0.801457,-0.482243,-0.410117
...,...,...,...
185,-0.062723,-0.482243,-0.326836
186,1.414746,0.369719,1.838455
192,1.414746,-0.482243,0.006285
194,-0.801457,-0.482243,-0.410117


In [24]:
lm = LinearRegression()

In [25]:
columns_to_plot = ("experience", "age", "gender", "played_hours")

players_pairplot = alt.Chart(players_scaled).mark_point().encode(
    alt.X(alt.repeat("row"), type="quantitative"),
    alt.Y(alt.repeat("column"), type="quantitative"),
).properties(
    width=200,
    height=200
).repeat(
    column=columns_to_plot,
    row=columns_to_plot
)
players_pairplot

alt.RepeatChart(...)

In [11]:
players_training, players_testing = train_test_split(
    players_filtered, test_size=0.25, random_state=2024
)
X_train_play = players_training[["experience", "gender", "age"]]
y_train_play = players_training["played_hours"]

X_test_play = players_testing[["experience", "gender", "age"]]
y_test_play = players_testing["played_hours"]

In [12]:
players_fit = lm.fit(X_train_play, y_train_play)
players_fit

LinearRegression()

In [30]:
lm_fit = lm.fit(X_train_play, y_train_play)
lm_fit

LinearRegression()

In [31]:
print(lm_fit.coef_)
print(lm_fit.intercept_)

[-2.17014437 10.26076585 -0.63728233]
26.334812887942178


In [32]:
play_predictions = lm.predict(X_train_play)
lm_rmse = mean_squared_error(y_train_play, play_predictions)**(1/2)
lm_rmse

np.float64(41.36276520514891)

In [34]:
play_predictions = lm.predict(X_test_play)
lm_rmspe = mean_squared_error(y_test_play, play_predictions)**(1/2)
lm_rmspe

np.float64(20.83961263101158)

In [36]:
#Training set 
train_pred = players_training.assign(predictions=lm_fit.predict(X_train_play))
scatterplot = alt.Chart(train_pred).mark_circle(opacity=0.4).encode(
    x1=alt.X("experience").title("Experience Level").scale(zero=False),
    x2=alt.X("age").title("Age of players").scale(zero=False),
    y=alt.Y("played_hours").title("Marathon time (in hours)").scale(zero=False)
)
marathon_plot = scatterplot + scatterplot.mark_line(color='black').encode(y="predictions")

,experience,gender,age,played_hours,predictions
4,2,0,21,0.1,8.611595
44,4,0,24,2.2,2.359460
59,4,0,38,0.2,-6.562493
96,1,0,19,0.1,12.056304
162,0,0,19,0.6,14.226449
...,...,...,...,...,...
61,2,0,20,1.5,9.248878
46,4,0,17,0.1,6.820436
0,0,0,9,30.3,20.599272
180,1,0,24,2.1,8.869893


In [39]:
#Testing set 
test_pred = players_testing.assign(predictions=lm_fit.predict(X_test_play))
test_pred

,experience,gender,age,played_hours,predictions
81,4,4,17,1.0,47.863499
29,4,0,18,0.1,6.183154
35,4,0,20,0.4,4.908589
42,1,0,24,0.3,8.869893
177,4,4,21,2.7,45.314370
24,1,1,21,0.7,21.042506
185,2,0,18,0.1,10.523442
82,3,1,37,0.2,6.505700
145,4,0,19,0.1,5.545871
31,4,0,21,0.1,4.271307
